In [2]:
using Oscar
using Combinatorics

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [27]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end

function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end

function findBarycenter(s,X)
    rays = rowMinors(Array(X.RAYS), s)
    dim=size(rays,2)
    bary=zeros(Polymake.Rational,dim,1)
    for i in 1:size(rays,1)
        bary+=rays[i,:]
    end
    bary=Polymake.common.primitive(bary)
    return bary
end

function toric_blowup(s, X, v)
    s = [i + 1 for i in s]
    if v==nothing
        v=findBarycenter(s,X)
    end
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = [coneList[i] for i in starIndex]
    rayMatrix = X.RAYS
    
    lattice = X.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
        rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
        for cone in rank_c_subcones
            new_cone = [i+1 for i in cone]
            if all((i -> i in t).(new_cone))
                push!(clStar, new_cone)
            end
        end
    end
    clStar = unique(clStar)
    
    n = size(rayMatrix, 1) + 1
    coneList = filter(x -> !(x in star), coneList)
    
    if length(s) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                push!(newCones, sort(push!(t, s[1])))
            end
        end
        finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
        return Polymake.fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            push!(newCones, push!(t, n))
        end
    end
    finalRays = vcat((X.RAYS),transpose(v))
    finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
    return Polymake.fulton.NormalToricVariety(INPUT_RAYS = finalRays, INPUT_CONES = finalCones)
end

function convertBool(B)
    out=[]
    for i in 1:size(B,1)
        if B[i]==true
           append!(out,i-1) 
        end
    end
    return out
end

convertBool (generic function with 1 method)

In [28]:
function getConeRays(coneRayIndices, rayMatrix)
    return convert(Matrix{Int}, tv.RAYS[findall(bool -> bool, coneRayIndices),:])
end

function getConeRank(coneRayIndices, rayMatrix)
    coneRays = rowMinors(rayMatrix,coneRayIndices)
    return rank(Matrix(coneRays))
end

function getDimension(X)
    return size(X.RAYS, 2)
end



function getConeFacesImproved(fan,cone,rayMatrix)
    lattice = fan.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    cone_faces=[]
    c = rank(Array(rowMinors(rayMatrix, cone))) - 1
    rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
    rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
    for subcone in rank_c_subcones
        new_cone = [i+1 for i in subcone]
        if all((i -> i in cone).(new_cone))
            push!(cone_faces, new_cone)
        end
    end 
    return cone_faces
end

function makeSimplicial(X)
    Y = copy(X)
    while (true)
        if Y.SIMPLICIAL==true
            break
        end
        #Maximal cones and ray matrix
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        badCone = nothing
        for i in 1:size(coneList,1)
            cone = coneList[i,:]
            if (getConeRank(cone, rayMatrix) != size(cone))
                badCone = cone
            end
        end
        if (badCone == nothing)
            # All cones are linearly independent
            break
        else
            # Find the first ray that is contained in more than one orbit
            # and subdivide at that ray, using toricBlowup
            
            # Get faces (need to replace this)
            edges = getConeFacesImproved(Y,badCone,rayMatrix)
            # Find the first ray that is contained in more than one orbit
            i = 1
            while count(r->(badCone[i] in r), edges) == 1
                i += 1
            end
            # Subdivide at the cone containing just that ray
            Y = toric_blowup(convertBool(badCone), Y,nothing)
            #Y = toric_blowup([badCone[i]], Y,nothing)
        end
        # Repeat this process until there are no more bad cones
        #break
    end
    return Y
end



makeSimplicial (generic function with 1 method)

In [29]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 
function makeSmooth(X)
    Y  = copy(X)
    while(true)
        coneList = convertIncidenceMatrix(Y.MAXIMAL_CONES)
        rayMatrix = Array(Y.RAYS)
        k = 1
        for coneSet in coneList
            S=size(coneSet)[1]
            coneRays=rowMinors(rayMatrix,coneSet)
            R=rank(coneRays)
            smoothCheck=Polymake.fan.check_fan_objects(Polymake.polytope.Cone(RAYS=coneRays)).SMOOTH_FAN
            if R != S || !smoothCheck
                break
            else
                k+=1
            end
        end
        if k == size(coneList,1)+1
            break
        end
        sigma=coneList[k]
        sigmaRays=slicematrix(rowMinors(rayMatrix,sigma))
        tau=0; tauRays=0; tauCone=0
        for subset in collect(powerset(sigma))
            if size(subset,1) > 1
                S=size(subset)[1]
                subsetRays=rowMinors(rayMatrix,subset)
                R=rank(subsetRays)
                subsetCone=Polymake.polytope.Cone(RAYS=subsetRays)
                smoothCheck=Polymake.fan.check_fan_objects(subsetCone).SMOOTH_FAN
                if R != S || !smoothCheck
                    tau=subset
                    tauRays=subsetRays
                    tauCone=subsetCone
                    break
                end 
            end
        end
        H=slicematrix(Matrix(tauCone.HILBERT_BASIS_GENERATORS[1]))
        rayIndex=0
        for i in 1:size(H,1)
            if !(H[i] in sigmaRays)
                rayIndex=i
                break
            end
        end
        if rayIndex==0
            Y=makeSimplicial(Y)
            break
        else
            blowupRay=H[rayIndex]
            tau=[i-1 for i in tau]
            Y=toric_blowup(tau,Y,blowupRay)
            break
        end
    end
    return Y
end

makeSmooth (generic function with 1 method)

In [36]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0 0;1 2 0;0 0 1;0 1 0; 1 1 1],INPUT_CONES=[[0,1,2],[0,2,3,4]])

type: NormalToricVariety

INPUT_CONES
	{0 1 2}
	{0 2 3 4}

INPUT_RAYS
	1 0 0
	1 2 0
	0 0 1
	0 1 0
	1 1 1

In [37]:
makeSmooth(X)

type: NormalToricVariety

INPUT_CONES
	{0 2 3}
	{0 2 4}
	{1 2 4}

INPUT_RAYS
	1 0 0
	1 2 0
	0 0 1
	0 1 0
	1 1 0